In [2]:
# Priority
* in processing chains add the relative orbit identifier to be able to extract it later on
* check if field border pixels were excluded in subsequent analyses

# To-Do & Future improvements
* corrects for thermal noise
    + either apply thermal noise correction & substract from original
    + or crop noise manually from LUT
    + or for simplicity assuming that contribution is negligible
    + testing multiple applications of thermal noise removal on images with prior addition of x to avoid clipping of values to 0
* polarimetric decomposition
* for GRD: compare & cross-validate with GRD product as created/provided by ESA

# Technical improvements
* install geopandas in pyton 3.6 (snap venv)

# Questions
* Is TOPSAR-Split required only to subset the scene or for all analyses to apply orbit files specifically to certain parts of the image?

## Setup & Imports

In [2]:
# set locale to make matplotlib import work properly
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# common imports
import glob
import matplotlib.colors as colors
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import subprocess
import time
import zipfile

# establish connection to SNAP & import corresponding libraries
snap_venv = 'C:\\Users\\felix\\.virtualenvs\\snap\\Lib'
sys.path.append(snap_venv)
import snappy
import jpy

# change some notebook settings
pd.options.display.max_colwidth = 80

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# define relevant paths
python_kernel = "C://Users/felix/.virtualenvs/snap/Scripts/python.exe"
s1_data_path = "D://Adv_Rs_project/01_data_raw/s1_slc"
work_dir = "C://Users/felix/github/sar_parcel_analyses"
wkt_file = os.path.join(work_dir, "01_data_raw", "test_area_processing.txt")

In [4]:
# define help function to retrieve info for operators
def snap_help(*operator):
    print(subprocess.Popen(['gpt', '-h', *operator], stdout=subprocess.PIPE, universal_newlines=True).communicate()[0])

# demonstration of help function
# snap_help("Polarimetric-Matrices")

## Read & Reorganise scenes

In [5]:
# organise products as consecutive 6-day-pairs from same relative orbit

# list all s1 products
s1_files = glob.iglob(os.path.join(s1_data_path, "*S1*.zip"))
s1_files = [os.path.basename(x) for x in list(s1_files)]

# compile infos about s1 products into comprehensive df
satellite, absolute_orbit, date_time, filename = ([] for i in range(4))

for i in s1_files:
    satellite.append(i.split("_")[0])
    absolute_orbit.append(i.split("_")[7])
    date_time.append(i.split("_")[5])
    filename.append(i)

df_s1 = pd.DataFrame({
    'Satellite' : satellite,
    'Absolute Orbit': absolute_orbit,
    'Time': date_time,
    'Filename': filename
})

# conversion of orbit specifications based on filenames
def convert_abs_rel_orbit(abs_orbit, satellite):
    if satellite == "S1A":
        rel_orbit = (int(abs_orbit) - 73) % 175 + 1
    if satellite == "S1B":
        rel_orbit = (int(abs_orbit) - 27) % 175 + 1
    return rel_orbit

df_s1.insert(2, "Relative Orbit", [convert_abs_rel_orbit(x,y) for x,y, in zip(df_s1["Absolute Orbit"], df_s1["Satellite"])])

# arrange df to retrieve consecutive pairs
df_s1["Time"] = pd.to_datetime(df_s1["Time"], format='%Y-%m-%dT%H:%M:%S')
df_s1 = df_s1.sort_values(["Relative Orbit", "Time"]).reset_index(drop=True)
S1_SLC_pairs = df_s1.groupby("Relative Orbit")["Filename"].transform(lambda x: zip(x, x.shift(-1)))
S1_SLC_pairs_tdelta = df_s1.groupby("Relative Orbit")["Time"].transform(lambda x: x.shift(-1) - x)
df_s1["SLC Pairs"] = S1_SLC_pairs
df_s1["Pair Timedelta"] = S1_SLC_pairs_tdelta

# check time deltas
long_delta = df_s1[df_s1["Pair Timedelta"] > pd.Timedelta(days=7)] 
print(f"Warning: The timedelta between scenes is not equal, {len(long_delta)} scene pair is more than 6 days apart.\n")

# print results
df_s1

,Satellite,Absolute Orbit,Relative Orbit,Time,Filename,SLC Pairs,Pair Timedelta
0,S1B,026320,44,2021-04-04 16:58:29,S1B_IW_SLC__1SDV_20210404T165829_20210404T165857_026320_032437_1DA7.zip,"(S1B_IW_SLC__1SDV_20210404T165829_20210404T165857_026320_032437_1DA7.zip, S1...",6 days 00:00:44
1,S1A,037391,44,2021-04-10 16:59:13,S1A_IW_SLC__1SDV_20210410T165913_20210410T165940_037391_046811_38EF.zip,"(S1A_IW_SLC__1SDV_20210410T165913_20210410T165940_037391_046811_38EF.zip, S1...",5 days 23:59:16
2,S1B,026495,44,2021-04-16 16:58:29,S1B_IW_SLC__1SDV_20210416T165829_20210416T165857_026495_0329CA_9CB3.zip,"(S1B_IW_SLC__1SDV_20210416T165829_20210416T165857_026495_0329CA_9CB3.zip, S1...",6 days 00:00:44
3,S1A,037566,44,2021-04-22 16:59:13,S1A_IW_SLC__1SDV_20210422T165913_20210422T165940_037566_046E25_93D0.zip,"(S1A_IW_SLC__1SDV_20210422T165913_20210422T165940_037566_046E25_93D0.zip, S1...",5 days 23:59:17
4,S1B,026670,44,2021-04-28 16:58:30,S1B_IW_SLC__1SDV_20210428T165830_20210428T165857_026670_032F67_4852.zip,"(S1B_IW_SLC__1SDV_20210428T165830_20210428T165857_026670_032F67_4852.zip, S1...",6 days 00:00:44
...,...,...,...,...,...,...,...
65,S1B,028996,95,2021-10-05 05:17:43,S1B_IW_SLC__1SDV_20211005T051743_20211005T051810_028996_0375C9_067D.zip,"(S1B_IW_SLC__1SDV_20211005T051743_20211005T051810_028996_0375C9_067D.zip, S1...",6 days 00:00:42
66,S1A,040067,95,2021-10-11 05:18:25,S1A_IW_SLC__1SDV_20211011T051825_20211011T051853_040067_04BE64_5A70.zip,"(S1A_IW_SLC__1SDV_20211011T051825_20211011T051853_040067_04BE64_5A70.zip, S1...",5 days 23:59:18
67,S1B,029171,95,2021-10-17 05:17:43,S1B_IW_SLC__1SDV_20211017T051743_20211017T051810_029171_037B32_8EF0.zip,"(S1B_IW_SLC__1SDV_20211017T051743_20211017T051810_029171_037B32_8EF0.zip, S1...",6 days 00:00:42
68,S1A,040242,95,2021-10-23 05:18:25,S1A_IW_SLC__1SDV_20211023T051825_20211023T051853_040242_04C47E_CF1D.zip,"(S1A_IW_SLC__1SDV_20211023T051825_20211023T051853_040242_04C47E_CF1D.zip, S1...",5 days 23:59:18


## Processing (Coherence, Polarimetric Parameters, Backscatter Intensity)

In [ ]:
# note that processing is done via snappy (see underlying .py scripts)
# notebook calls .py scripts for each scene/scene pair
# superior to direct implementation in notebook as memory blocked by SNAP is released after each cycle

In [8]:
# Coherence processing
out_coh = os.path.join(work_dir, "02_data_processed", "coherence")

for idx, pair in enumerate(df_s1["SLC Pairs"]):
    # only proceed if pair is valid (no nan)
    if pair[0] == pair[0] and pair[1] == pair[1]:  
        file_1 = os.path.join(s1_data_path, pair[0])
        file_2 = os.path.join(s1_data_path, pair[1])
        # try workflow several times in case of failure (to account for java overload errors occuring from time to time)
        attempts = 0
        done = False
        while not done and attempts < 2:
            try:
                pipeline_out = subprocess.check_output(
                    [python_kernel, 
                    'coherence_processing.py', 
                    '-file_I', file_1, 
                    '-file_II', file_2,
                    '-out_dir', out_coh,
                    '-snap_env', snap_venv,
                    '-aoi_wkt', wkt_file,
                    '-crs', "32633"], 
                    stderr=subprocess.STDOUT
                )
                print(f"Product iteration {idx} sucessfully generated ({time.ctime()}, attempt {attempts+1}).")
                done = True
            except:
                attempts += 1
                if attempts == 2:
                    print(f"Warning: Product in iteration {idx} could not be generated despite several attempts.")

Product iteration 0 sucessfully generated (Thu Jul 14 20:21:22 2022, attempt 1).


In [6]:
# Intensity processing
out_int = os.path.join(work_dir, "02_data_processed", "intensity")

for idx, s1_file in enumerate(df_s1["Filename"]):
    file = os.path.join(s1_data_path, s1_file)
    # try workflow several times in case of failure (to account for java overload errors occuring from time to time)
    attempts = 0
    done = False
    while not done and attempts < 3:
        try:
            pipeline_out = subprocess.check_output(
                    [python_kernel, 
                    'intensity_processing.py', 
                    '-s1_file', file, 
                    '-out_dir', out_int,
                    '-snap_env', snap_venv,
                    '-aoi_wkt', wkt_file,
                    '-crs', "32633"], 
                    stderr=subprocess.STDOUT
                )
            print(f"Product iteration {idx} sucessfully generated ({time.ctime()}, attempt {attempts+1}).")
            done = True
        except:
            attempts += 1
            if attempts == 3:
                print(f"Warning: Product in iteration {idx} could not be generated despite several attempts.")

Product iteration 0 sucessfully generated (Thu Jul 14 17:37:34 2022, attempt 1).
Product iteration 1 sucessfully generated (Thu Jul 14 17:41:28 2022, attempt 1).
Product iteration 2 sucessfully generated (Thu Jul 14 17:45:49 2022, attempt 1).
Product iteration 3 sucessfully generated (Thu Jul 14 17:50:51 2022, attempt 2).


In [ ]:
# Polarimetric decomposition processing
out_pol = os.path.join(work_dir, "02_data_processed", "polarimetry")

for idx, s1_file in enumerate(df_s1["Filename"]):
    file = os.path.join(s1_data_path, s1_file)
    # try workflow several times in case of failure (to account for java overload errors occuring from time to time)
    attempts = 0
    done = False
    while not done and attempts < 3:
        try:
            pipeline_out = subprocess.check_output(
                    [python_kernel, 
                    'polarimetry_processing.py', 
                    '-s1_file', file, 
                    '-out_dir', out_pol,
                    '-snap_env', snap_venv,
                    '-aoi_wkt', wkt_file,
                    '-crs', "32633"], 
                    stderr=subprocess.STDOUT
                )
            print(f"Product iteration {idx} sucessfully generated ({time.ctime()}, attempt {attempts+1}).")
            done = True
        except:
            attempts += 1
            if attempts == 3:
                print(f"Warning: Product in iteration {idx} could not be generated despite several attempts.")